In [5]:
import pandas as pd

In [6]:
data20 = pd.read_csv("debate_transcripts_v3_2020-02-26.csv", encoding='ISO-8859-1')

In [15]:
candidate_names = """Andrew Yang
John Delaney
Elizabeth Warren
Beto O'Rourke
Pete Buttigieg
Bernie Sanders
Steve Bullock
Michael Bennet
Amy Klobuchar
Kamala Harris
Tulsi Gabbard
Marriane Williamson
Julian Castro
Cory Booker
Joe Biden
Jay Inslee
John Hickenlooper
Tom Steyer
Michael Bloomberg
Eric Swalwell
Bill De Blasio
Kirsten Gillibrand
Seth Moulton
Tim Ryan"""
candidate_names = candidate_names.split("\n")

In [9]:
count = data20['speech'].str.split().apply(len)
for i in range(5,25):
    print(i)
    print(data20[count >= i].shape[0])
    print('')

5
4657

6
4407

7
4192

8
4037

9
3885

10
3772

11
3645

12
3548

13
3479

14
3414

15
3335

16
3275

17
3205

18
3145

19
3102

20
3048

21
3001

22
2958

23
2928

24
2889

